In [1]:
from nltk import sent_tokenize
import re
import json
from pydub import AudioSegment
from aeneas.executetask import ExecuteTask
from aeneas.task import Task

In [2]:
# Step 1: read script from script.txt, remove newlines, and splitting into sentences

AUDIO_CHAPTER_FILE = "DCLG7C9.wav"  # Path to the wav audio file (can be relative)
SCRIPT_CHAPTER_FILE = "script.txt" # Path to the script file (can be relative) 
OUTPUT_FILE = "output.json"

r = r'\n+'
with open(SCRIPT_CHAPTER_FILE,"r") as script:
    text = script.read()
new_text = re.sub(pattern=r,repl=" ",string=text)

sentences = sent_tokenize(new_text)

print("Number of sentences:", len(sentences))

with open(SCRIPT_CHAPTER_FILE,"w") as script:
    script.writelines("\n".join(sentences))

Number of sentences: 291


Step 2: 
Download Audacity from <https://www.audacityteam.org/>, open the audio chapter, double check with the script file to find spelling errors, etc. Also remove all the parts with music. Exported the clean audio to wav format

In [3]:
# Step 3: split the audio using the script with aeneas

config_string = "task_language=vie|os_task_file_format=json|is_text_type=plain|task_adjust_boundary_algorithm=rate|task_adjust_boundary_rate_value=17|task_adjust_boundary_nonspeech_min=0.1|task_adjust_boundary_nonspeech_string=REMOVE"
task = Task(config_string=config_string)
task.audio_file_path_absolute = AUDIO_CHAPTER_FILE 
task.text_file_path_absolute = SCRIPT_CHAPTER_FILE 
task.sync_map_file_path_absolute = OUTPUT_FILE 
ExecuteTask(task).execute() # execute
task.output_sync_map_file() # write the sync map to output file

'output.json'

In [4]:
# Step 4: convert unicode to string
# Since aeneas use python 2, and python 2 doesn't have string type, we need to convert it to python 3 string

with open(OUTPUT_FILE,"r") as output:
    arr = json.load(output)
json.dump(fp=open(OUTPUT_FILE,"w"),obj=arr,ensure_ascii=False)

In [5]:
import os
book = AudioSegment.from_wav(AUDIO_CHAPTER_FILE)

# Step 5: Splitting the audio book into sentences
if not os.path.exists("splitted"): # Make splitted directory
    os.makedirs("splitted") 
    
os.chdir("splitted/") 

for i in range (0,len(arr["fragments"])):
    fragment = arr["fragments"][i]
    begin = float(fragment["begin"])*1000
    end = float(fragment["end"])*1000
    clip = book[begin:end]
    clip.export("{}.wav".format(fragment["id"][-3:]),format="wav")

os.chdir("..")

Step 6:  
Navigate to splitted folder, and do the last manual check of the splitted audios file to make sure they match the script. Every splitted audio file should contain **1 sentence**